In [1]:

year = 104    #110年或104年
rush_hour = '晨峰'    #'晨峰'或'昏峰'

inputLocation = f"{year}_現況_{rush_hour}/"
def get_capacity(row):
            class_num = row['CLASS_110']
            lanes = row['LANES_110']
            if (class_num, lanes) in cap_dict:
                return cap_dict[(class_num, lanes)]
            else:
                print(f"Warning: No capacity found for class {class_num} and lanes {lanes}. Using default value 0.")
                return 0

def savetoShapefile():
    cap_dict = {}
    #input
    # CLASS,L1,L2,L3,L4,L5,L6
    # 1, 2300, 4300, 6300, 8300, 10300, 12300
    #disct: (class,lanes)=capacity eg (39,6)=9420
    with open('TRTS4S_Capacity.csv', 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:  # Skip the header line
            parts = line.strip().split(',')
            class_num = int(parts[0])
            for i in range(1, len(parts)):
                lanes = i
                capacity = int(parts[i])
                cap_dict[(class_num, lanes)] = capacity
    ue_results = pd.read_csv(f"{year}_{scenario}_{rush_hour}_UE_results.dat", sep="\t")
    gdf = gpd.read_file(f"Network/Net_modified_{scenario}_{rush_hour}調撥.shp")
    # Rename columns to match
    ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)
    # Merge results back into the shapefile
    gdf = gdf.merge(ue_results[["A", "B", "capacity", "UE_travelTime", "UE_flow"]], on=["A", "B"], how="left")
    # Ensure 'capacity' is filled based on the class and lanes
    gdf['capacity'] = gdf.apply(get_capacity, axis=1)
    # Calculate V/C ratio
    gdf["V/C"] = gdf["UE_flow"] / gdf["capacity"]
    # Calculate assigned speed
    gdf["assigned_speed"] = gdf["LENGTH"] / gdf["UE_travelTime"] * 60 #km/min->km/h
    # Save updated shapefile
    gdf.to_file(f"{year}_{scenario}_{rush_hour}_UE_results.shp")

savetoShapefile() 

FileNotFoundError: [Errno 2] No such file or directory: 'TRTS4S_Capacity.csv'